# Calculo de las metricas en python

### Importacion de las librerias necesarias

In [12]:
import json
import re
import pandas as pd
import matplotlib.pyplot as plt

### Lectura del documento

In [13]:
# Ruta del archivo con los datos obtenidos por el "Tracking system"
file_path = './RESULT.json'

In [14]:
# Abrir el archivo json y cargarlo en la variable data
with open(file_path) as f:
    data = json.load(f)

# Se ordena la información del json en base el timestamp
sorted_data = sorted(data, key=lambda x: x['timestamp'])

In [17]:
# Longitud de cada sesion en milisegundos
gameSessionLengthMs = []
for currentEvent in sorted_data:
    if currentEvent['typeEvent'] == "LoginEvent":
        tsSessionStart = currentEvent['timestamp']
        r = re.search(r'\d+', tsSessionStart)
        numeric_tsSessionStart = int(r.group())
    if currentEvent['typeEvent'] == "LogoutEvent":
        tsSessionEnd = currentEvent['timestamp']
        r = re.search(r'\d+', tsSessionEnd)
        numeric_tsSessionEnd = int(r.group())
        gameSessionLengthMs.append(numeric_tsSessionEnd- numeric_tsSessionStart)

s = pd.Series(gameSessionLengthMs)
#print(s.describe())

count         1.0
mean     229048.0
std           NaN
min      229048.0
25%      229048.0
50%      229048.0
75%      229048.0
max      229048.0
dtype: float64


In [18]:
s

0    229048
dtype: int64

In [ ]:
index = 0
num_events = len(sorted_data)

while index < num_events:
    currentEvent = sorted_data[index]
    consumeEvent = currentContext.parseEvent(currentEvent)
    if consumeEvent:
        index+=1

In [ ]:
def parseEvent(event):
    